In [ ]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import os
import torch as t
import torch.optim as optim
from sklearn.metrics import roc_auc_score
import time
import pickle
import scipy.sparse as sp
import argparse

In [ ]:
if t.cuda.is_available():
    print("CUDA GPU is available")
else:
    print("CUDA GPU is not available")


CUDA GPU is available


In [ ]:
def negative_sample(datype, data, user_history_dict, user_positive_ratings, item_set, ratio=1):
    if datype == 'train':
        history = user_history_dict
    else:
        history = user_positive_ratings
    split_data = []
    for i in data:
        user_index = i[0]
        pos_item_index = i[1]
        negative_set = item_set - set(history[user_index])
        for neg_item_index in np.random.choice(list(negative_set), size=ratio, replace=False):
            split_data.append([user_index, pos_item_index, neg_item_index])
    np.save('../data/' + DATASET +
            '/' + datype + '_data.npy', split_data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
RATING_FILE_NAME = dict({'movie': 'ratings.dat', 'book': 'BX-Book-Ratings.csv', 'news': 'ratings.txt', 'music': 'user.dat'})
SEP = dict({'movie': '::', 'book': '::', 'music': '\t'})
THRESHOLD = dict({'movie': 4, 'book': 0, 'news': 0, 'music': 0})

In [ ]:
def read_item_index_to_entity_id_file():
    file = '/content/drive/MyDrive/Code/CGAT-main/data/DATASET/item_index2entity_id.txt'
    print('reading item index to entity id file: ' + file + ' ...')
    i = 0
    for line in open(file, encoding='utf-8').readlines():
        item_index = line.strip().split('\t')[0]
        satori_id = line.strip().split('\t')[1]
        item_index_old2new[item_index] = i
        entity_id2index[satori_id] = i
        i += 1

In [ ]:
def convert_rating():
    print('reading rating file ...')
    file = '/content/drive/MyDrive/Code/CGAT-main/data/DATASET/ratings.dat'
    print(file)
    user_pos_ratings = dict()

    for line in open(file, encoding='utf-8').readlines()[1:]:
        array = line.strip().split('::')
        item_index_old = array[1]
        if item_index_old not in item_index_old2new:  # the item is not in the final item set
            continue
        item_index = item_index_old2new[item_index_old]

        user_index_old = int(array[0])

        rating = float(array[2])
        if rating >= 0:
            if user_index_old not in user_pos_ratings:
                user_pos_ratings[user_index_old] = set()
            user_pos_ratings[user_index_old].add(item_index)
    print('converting rating file ...')

    user_cnt = 0
    user_index_old2new = dict()
    rating_all = []
    # store the pairs of user and positive or negetive(unwatched) item
    for user_index_old, pos_item_set in user_pos_ratings.items():
        if user_index_old not in user_index_old2new:
            user_index_old2new[user_index_old] = user_cnt
            user_cnt += 1
        user_index = user_index_old2new[user_index_old]
        for item in pos_item_set:
            rating_all.append([user_index, item])

    np.save('../data/' + 'DS' + '/ratings_final.npy', rating_all)
    item_set = set(item_index_old2new.values())
    data_split(item_set)
    print('number of users: %d' % user_cnt)
    print('number of items: %d' % len(item_set))

In [ ]:
def data_split(item_set):
    rating_file = '/content/drive/MyDrive/Code/CGAT-main/data/music/ratings_final.npy'
    rating_np = np.load(rating_file)
    user_positive_items = dict()
    for rate in rating_np:
        user = rate[0]
        item = rate[1]
        if user not in user_positive_items:
            user_positive_items[user] = []
        user_positive_items[user].append(item)
    eval_ratio = 0.2
    test_ratio = 0.2
    n_ratings = rating_np.shape[0]

    eval_indices = np.random.choice(n_ratings, size=int(
        n_ratings * eval_ratio), replace=False)
    left = set(range(n_ratings)) - set(eval_indices)
    test_indices = np.random.choice(list(left), size=int(
        n_ratings * test_ratio), replace=False)
    train_indices = list(left - set(test_indices))
    user_history_dict = dict()
    for i in train_indices:
        user = rating_np[i][0]
        item = rating_np[i][1]
        if user not in user_history_dict:
            user_history_dict[user] = []
        user_history_dict[user].append(item)
    train_indices = [i for i in train_indices if rating_np[i]
                     [0] in user_history_dict]
    eval_indices = [i for i in eval_indices if rating_np[i]
                    [0] in user_history_dict]
    test_indices = [i for i in test_indices if rating_np[i]
                    [0] in user_history_dict]
    train_data = rating_np[train_indices]
    eval_data = rating_np[eval_indices]
    test_data = rating_np[test_indices]

    np.save('../data/' + 'DS' + '/user_history_dict.npy', user_history_dict)

    negative_sample('train', train_data, user_history_dict,
                    user_positive_items, item_set, ratio=5)
    negative_sample('eval', eval_data, user_history_dict,
                    user_positive_items, item_set)
    negative_sample('test', test_data, user_history_dict,
                    user_positive_items, item_set)

In [ ]:
def convert_kg():
    if not os.path.exists('../data/DATASET/'):
      os.makedirs('../data/DATASET/')
    print('converting kg file ...')
    entity_cnt = len(entity_id2index)
    relation_cnt = 0

    # writer = open('../data/' + DATASET +
    #               '/kg_final.txt', 'w', encoding='utf-8')

    files = []
    files.append(open('/content/drive/MyDrive/Code/CGAT-main/data/music/kg.txt', encoding='utf-8'))
    kg_final = []

    for file in files:
        for line in file:
            array = line.strip().split('\t')
            head_old = array[0]
            relation_old = array[1]
            tail_old = array[2]

            if head_old not in entity_id2index:
                entity_id2index[head_old] = entity_cnt
                entity_cnt += 1
            head = entity_id2index[head_old]

            if tail_old not in entity_id2index:
                entity_id2index[tail_old] = entity_cnt
                entity_cnt += 1
            tail = entity_id2index[tail_old]

            if relation_old not in relation_id2index:
                relation_id2index[relation_old] = relation_cnt
                relation_cnt += 1
            relation = relation_id2index[relation_old]

            kg_final.append([head, relation, tail])
        file.close()

    np.save('../data/' + DS + '/kg_final.npy', kg_final)

    print('number of entities (containing items): %d' % entity_cnt)
    print('number of relations: %d' % relation_cnt)


In [ ]:
adj_entity_gb_p = 'adj_entity_gb'
def load_data(args):
    train_data, eval_data, test_data, user_history_dict, n_user, n_item = load_rating(args)
    n_entity, n_relation, kg, adj_enlc, adj_relc, adj_engb = load_kg(args, n_item)
    user_history_dict = fix_userhist(args, user_history_dict)
    print('Loaded data')
    return train_data, eval_data, test_data, n_entity, n_relation, n_user, n_item, kg, adj_enlc, adj_relc, user_history_dict, adj_engb

In [ ]:
def load_rating(args):
    print('reading rating file ...')

    # reading rating file
    train_file = '/content/drive/MyDrive/Code/CGAT-main/data/music/train_data.npy'
    eval_file = '/content/drive/MyDrive/Code/CGAT-main/data/music/eval_data.npy'
    test_file = '/content/drive/MyDrive/Code/CGAT-main/data/music/test_data.npy'
    train_np = np.load(train_file)
    test_np = np.load(test_file)
    eval_np = np.load(eval_file)

    user_history_dict = dict()
    for i in train_np:
        user = i[0]
        item = i[1]
        if user not in user_history_dict:
            user_history_dict[user] = set()
        user_history_dict[user].add(item)
    for user, history_item in user_history_dict.items():
        user_history_dict[user] = list(history_item)

    n_user = max(set(train_np[:, 0]) | set(
        test_np[:, 0]) | set(eval_np[:, 0])) + 1
    n_item = max(set(train_np[:, 1]) | set(train_np[:, 2]) | set(
        test_np[:, 1]) | set(test_np[:, 2]) | set(eval_np[:, 1]) | set(eval_np[:, 2])) + 1
    interaction = int(len(train_np) / 5) + len(test_np) + len(eval_np)

    print('n_user=%d, n_item=%d' % (n_user, n_item))
    print('n_interaction=%d' % interaction)
    return train_np, eval_np, test_np, user_history_dict, n_user, n_item


In [ ]:
def load_kg(args, n_item):
    print('reading KG file ...')
    # reading kg file

    kg_np = np.load('/content/drive/MyDrive/Code/CGAT-main/data/music/kg_final.npy')

    n_entity = len(set(kg_np[:, 0]) | set(kg_np[:, 2]))
    n_relation = len(set(kg_np[:, 1]))
    kg_entity, kg_relation, adj_sp = construct_kg(args, kg_np, n_relation, n_entity)
    adj_entity, adj_relation = construct_adj(args, kg_entity, kg_relation, n_item)
    adj_enbro = construct_adj_gb(args, kg_entity, n_item, adj_sp)
    kg_train = '/content/drive/MyDrive/Code/CGAT-main/data/music/kg_train.npy'

    if os.path.exists(kg_train + '.npy'):
        kg_np_ne = np.load(kg_train + '.npy')
    else:
        tail_entity = kg_np[:, 2]
        neg_entity = np.array([np.random.choice(np.delete(np.arange(
            n_entity), tail_entity[i], axis=0), 1) for i in range(len(tail_entity))])  # xiugai
        kg_np_ne = np.concatenate((kg_np, neg_entity), 1)
        np.save(kg_train + '.npy', kg_np_ne)
    print('number of entity ', n_entity)
    print('number of relations', n_relation)
    print('number of triples', len(kg_np))
    return n_entity, n_relation, kg_np_ne, adj_entity, adj_relation, adj_enbro

In [ ]:
def construct_kg(args, kg_np, num_relation, n_entity):
    print('constructing knowledge graph...')
    kg_entity = dict()
    kg_relation = dict()
    for head, relation, tail in kg_np:
        if head not in kg_entity:
            kg_entity[head] = []
            kg_relation[head] = []
        kg_entity[head].append(tail)
        kg_relation[head].append(relation)
        if tail not in kg_entity:
            kg_entity[tail] = []
            kg_relation[tail] = []
        kg_entity[tail].append(head)
        kg_relation[tail].append(relation + num_relation)
    a_rows = kg_np[:, 0]
    a_cols = kg_np[:, 2]
    a_vals = [1] * len(a_rows)
    c_vals = [1] * n_entity
    c_rows = [i for i in range(n_entity)]
    c_cols = c_rows
    a_adj = sp.coo_matrix((a_vals, (a_rows, a_cols)), shape=(n_entity, n_entity))
    b_adj = sp.coo_matrix((a_vals, (a_cols, a_rows)),shape=(n_entity, n_entity))
    c_adj = sp.coo_matrix((c_vals, (c_rows, c_cols)),shape=(n_entity, n_entity))
    adj = (a_adj + b_adj + c_adj).tolil()

    return kg_entity, kg_relation, adj




In [ ]:
def construct_adj(args, kg_entity, kg_relation, item_num):
    # each line of adj_entity stores the sampled neighbor entities for a given entity
    # each line of adj_relation stores the corresponding sampled neighbor relations
    print('constructing local neighbor entities of items...')
    adj_entity = np.zeros([item_num, args.n_neighbor], dtype=np.int64)
    adj_relation = np.zeros([item_num, args.n_neighbor], dtype=np.int64)
    for entity in range(item_num):
        neighbors = kg_entity[entity]
        n_neighbors = len(neighbors)
        if n_neighbors >= args.n_neighbor:
            sampled_indices = np.random.choice(
                list(range(n_neighbors)), size=args.n_neighbor, replace=False)
        else:
            sampled_indices = np.random.choice(
                list(range(n_neighbors)), size=args.n_neighbor, replace=True)
        adj_entity[entity] = np.array(
            [kg_entity[entity][i] for i in sampled_indices])
        adj_relation[entity] = np.array(
            [kg_relation[entity][i] for i in sampled_indices])

    return adj_entity, adj_relation

In [ ]:
def construct_adj_gb(args, kg_entity, item_num, adj_sp):
    print('constructing global neighbor entities of items...')
    file = '/content/drive/MyDrive/Code/CGAT-main/data/music/adj_entity_gb.npy'
    if os.path.exists(file):
        adj_entity_gb = np.load(file)
        adj_entity_gb = adj_entity_gb[:, -args.n_neighbor:]
    else:
        number = 50
        adj_entity_gb = np.zeros([item_num, number], dtype=np.int64)
        for item in range(item_num):
            adj_entity_gb[item] = depth_search(args, kg_entity, item, adj_sp, number)
        np.save(file + '.npy', adj_entity_gb)
        adj_entity_gb = adj_entity_gb[:, -args.n_neighbor:]
    return adj_entity_gb

In [ ]:
def fix_userhist(args, user_history_dict):
    for user in user_history_dict:
        n_history = len(user_history_dict[user])
        replace = n_history < args.n_memory
        sampled_indices = np.random.choice(
            n_history, size=args.n_memory, replace=replace)
        user_history_dict[user] = [user_history_dict[user][i]
                                   for i in sampled_indices]
    return user_history_dict

In [ ]:
from collections import Counter
def depth_search(args, kg_entity, entity, adj_sp, number):
    adj_entity = []
    adj_entity.append(entity)
    for i in range(15):  # 2 5 10 15 20 25
        temp = entity
        for j in range(8):  # 4 8 12 16 20 24
            neighbors = np.array(kg_entity[temp])
            # neighbors = np.random.choice(neighbors, size=)
            if j == 0:
                probly = np.ones(len(neighbors)) * (1 / len(neighbors))
            else:
                probly = np.ones(len(neighbors)) * 0.8
                index = adj_sp[adj_entity[-2], neighbors].nonzero()[1]
                probly[index] = 0.2
            probly = probly / np.sum(probly)
            pick_nei = np.random.choice(neighbors, size=1, p=list(probly))[0]
            adj_entity.append(pick_nei)
            temp = adj_entity[-1]
    adj_entity = adj_entity[1:]
    a = Counter(adj_entity)
    if [entity] != list(a):
        del a[entity]
    b = a.most_common(number)
    adj_entity = np.array([i[0] for i in b])
    leng = len(adj_entity)
    if leng < number:
        sampled_indices = np.random.choice(
            list(range(leng)), size=number - leng, replace=True)
        adj_entity = np.hstack((adj_entity, adj_entity[sampled_indices]))
    return np.array(adj_entity)[::-1]

In [ ]:
if __name__ == '__main__':
    np.random.seed(555)

    parser = argparse.ArgumentParser()
    parser.add_argument('-d', '--dataset', type=str,
                        default='DATASET', help='which dataset to preprocess')
    parser.add_argument('-f')
    args = parser.parse_args()
    DATASET = args.dataset
    entity_id2index = dict()
    relation_id2index = dict()
    item_index_old2new = dict()
    read_item_index_to_entity_id_file()
    convert_kg()
    convert_rating()
    print('done')

reading item index to entity id file: /content/drive/MyDrive/Code/CGAT-main/data/DATASET/item_index2entity_id.txt ...
converting kg file ...


NameError: ignored

In [ ]:
import argparse
import random
import torch as t

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='music',help='which dataset to use')
parser.add_argument('--dim', type=int, default=64,help='dimension of entity and relation embeddings')
parser.add_argument('--l2_weight_rs', type=float, default=0.00005,help='weight of the l2 regularization term')
parser.add_argument('--lr_rs', type=float,default=0.01, help='learning rate')
parser.add_argument('--batch_size', type=int, default=512, help='batch size')
parser.add_argument('--n_epochs', type=int, default=50,help='the number of epochs')
parser.add_argument('--n_memory', type=int, default=16,help='fixed size of user historical items')
parser.add_argument('--use_cuda', type=bool, default=True,help='use cuda.')
parser.add_argument('--n_neighbor', type=int, default=4,help='fixed size of neighbor entities')
parser.add_argument('--kg_weight', type=float,default=0.0001, help='weight of regularization')
parser.add_argument('--dropout', type=float, default=0.3,help='Dropout rate')
parser.add_argument('-f')

np.random.seed(2019)
random.seed(2019)
t.manual_seed(2019)
t.cuda.manual_seed_all(2019)
args = parser.parse_args()

show_loss = True
show_topk = True
args = parser.parse_args()

show_loss = True
show_topk = True
data_info=load_data(args)


MODEL

In [ ]:
import torch as t
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import numpy as np


class CGAT(nn.Module):
    def __init__(self, args, n_entity, n_relation, n_users, n_items):
        super().__init__()
        self.args = args
        self.entity_embedding = nn.Embedding(n_entity, args.dim)
        self.relation_embedding = nn.Embedding(
            2 * n_relation + 1, args.dim)
        self.user_embedding = nn.Embedding(n_users, args.dim)
        # rs_model
        self.linear_u_mlp = nn.Linear(3 * args.dim, args.dim)
        self.weight_re = nn.Linear(2 * args.dim, args.dim, bias=False)
        self.weight_agg = nn.Linear(2 * args.dim, args.dim)
        self.gate = nn.Parameter(t.FloatTensor(args.dim))
        self.graph_att = nn.Linear(2 * args.dim, args.dim)
        self.item_att = nn.Linear(4 * args.dim, 1)
        self.rnn = nn.GRU(args.dim, args.dim, batch_first=True)
        self.trans_att = nn.Linear(args.dim, args.dim)
        self.user_specific = True
        if not self.user_specific:
            self.weight_nouser = nn.Linear(args.dim, 1, bias=False)

        self.Dropout = nn.Dropout(args.dropout)
        init.zeros_(self.gate.data)
        init.xavier_uniform_(self.relation_embedding.weight)

    # (batch, n_memories, dims)
    def forward(self, user_indices, entity_lc, relation_lc, entity_gb, kg):
        # set_trace()
        user_global_em = self.user_embedding(
            user_indices)  # (batch, embeddings)  user global embedding
        entity_hislc_em = self.entity_agg(
            entity_lc[0], relation_lc[0], user_global_em)  # local embedding of user historical items
        # global embedding of items (batch, n_memory, embeddings)
        entity_hisgb_em = self.entity_agg(entity_gb[0], None, None)
        # aggregate the local embedding and global embedding of items
        gate = t.sigmoid(self.gate)
        item_history_em = gate.expand_as(
            entity_hislc_em) * entity_hislc_em + (1 - gate).expand_as(entity_hisgb_em) * entity_hisgb_em
        item_history_em = t.cat((self.self_vectors, item_history_em), -1)

        if len(item_history_em) == 1:
            item_history_em = item_history_em.expand(
                len(entity_lc[1][0]), item_history_em.shape[1], item_history_em.shape[2])
            user_global_em = user_global_em.expand(
                len(entity_lc[1][0]), user_global_em.shape[1])
        pos_enlc_em = self.aggregate_lc(
            entity_lc[1], relation_lc[1], user_global_em)  # (batch, dims)  local embedding of candidate items
        pos_engb_em = self.aggregate_gb(entity_gb[1])
        pos_item_em = gate.expand_as(pos_enlc_em) * pos_enlc_em + (
            1 - gate).expand_as(pos_engb_em) * pos_engb_em
        pos_item_em = t.cat((self.self_vectors, pos_item_em), -1)
        userp_lc_em = self.rs_attention(
            item_history_em, pos_item_em)  # (batch, dims)
        userp_em = t.cat(
            (user_global_em, userp_lc_em), 1)  # (batch, 2*dims)
        userp_em = t.relu(
            self.linear_u_mlp(self.Dropout(userp_em)))
        userp_em = self.Dropout(userp_em)
        userp_em = F.normalize(userp_em, dim=-1)
        userp_em = t.cat((user_global_em, userp_em),
                         1)  # (batch, 2 * dims)
        pos_score = t.sum(userp_em * pos_item_em, 1)  # (batch,)

        if entity_lc[2] is not None:
            neg_enlc_em = self.aggregate_lc(
                entity_lc[2], relation_lc[2], user_global_em)  # (batch, dims)
            neg_engb_em = self.aggregate_gb(entity_gb[2])
            neg_item_em = gate.expand_as(neg_enlc_em) * neg_enlc_em + (
                1 - gate).expand_as(neg_engb_em) * neg_engb_em
            neg_item_em = t.cat((self.self_vectors, neg_item_em), -1)
            usern_lc_em = self.rs_attention(
                item_history_em, neg_item_em)  # (batch, dims)
            usern_em = t.cat(
                (user_global_em, usern_lc_em), 1)  # (batch, 2*dims)
            usern_em = t.relu(
                self.linear_u_mlp(self.Dropout(usern_em)))  # (batch, dims)
            usern_em = self.Dropout(usern_em)
            usern_em = F.normalize(usern_em, dim=1)
            usern_em = t.cat((user_global_em, usern_em), 1)
            neg_score = t.sum(usern_em * neg_item_em, 1)  # (batch,)
        else:
            neg_score = t.zeros(1).cuda()
        rs_loss = -t.mean(t.log(t.sigmoid(pos_score - neg_score)))

        # knowledge graph loss for regularization
        if kg[0] is not None:
            head_em = self.entity_embedding(kg[0])
            relation_em = self.relation_embedding(kg[1])
            tail_em = self.entity_embedding(kg[2])
            tail_ne_em = self.entity_embedding(kg[3])

            nere_vectors = t.cat((tail_em, relation_em), 1)
            nere_ne_vectors = t.cat(
                (tail_ne_em, relation_em), 1)  # (batch, 2dims)
            # (batch, dims)
            nere_vectors = self.weight_re(nere_vectors)
            nere_ne_vectors = self.weight_re(nere_ne_vectors)

            score_kge = t.sum((head_em - nere_vectors).pow(2), 1)
            score_kge_ne = t.sum((head_em - nere_ne_vectors).pow(2), 1)
            kg_loss = -t.mean(t.log(t.sigmoid(score_kge_ne - score_kge)))
        else:
            kg_loss = t.zeros(1).cuda()
        all_loss = rs_loss + self.args.kg_weight * kg_loss

        return pos_score, neg_score, all_loss

    # aggregate local embeddings of items
    def entity_agg(self, entity_his, relation_his, user_global_em):
        batch = entity_his[0].shape[0]
        # (batch * n_memories, 1), (batch * n_memories, neighbor)
        entity_his = [i.reshape([-1, i.shape[-1]]) for i in entity_his]
        if relation_his is not None:
            relation_his = relation_his.reshape(
                [-1, self.args.n_neighbor])  # (batch*n_memories, neighbor)
            user_em_att = user_global_em.unsqueeze(1).expand(
                batch, self.args.n_memory, self.args.dim)  # (batch, n_memories, dims)
            # (batch*n_memories, dims)
            user_em_att = user_em_att.contiguous().view(-1, self.args.dim)
            entity_his_vec = self.aggregate_lc(
                entity_his, relation_his, user_em_att)  # (batch*n_memories, dims)
            self.self_vectors = self.self_vectors.contiguous().view(
                [batch, self.args.n_memory, self.args.dim])
        else:
            entity_his_vec = self.aggregate_gb(entity_his)
        entity_his_vec = entity_his_vec.contiguous().view(
            [batch, self.args.n_memory, self.args.dim])  # (batch, n_memories, dims)
        return entity_his_vec

    def aggregate_lc(self, entities, relations, users_embedding):
        # (batch, 1, dims), (batch, neigh, dims)...
        entity_vectors = [self.entity_embedding(i) for i in entities]
        # (batch, neigh, dims)
        relation_vectors = self.relation_embedding(relations)
        self.self_vectors = entity_vectors[0].squeeze(1)  # (batch, dims)
        # (batch, neigh, dims)
        neighbor_vectors = entity_vectors[1]
        # (batch, neigh, 2dims)
        nere_vectors = t.cat((neighbor_vectors, relation_vectors), 2)
        # (batch, neigh, dims)
        nere_vectors = self.weight_re(nere_vectors)
        user_att = t.relu(self.trans_att(users_embedding))
        vector = self.SumAttention(self_vectors=self.self_vectors, neres=nere_vectors,
                                   user_embeddings=user_att, neighbors=neighbor_vectors, user_specific=self.user_specific)
        vector = self.Dropout(vector)
        vector = F.normalize(vector)
        return vector

    def aggregate_gb(self, entities):
        # (batch, 1, dims), (batch, neigh, dims)
        entity_vectors = [self.entity_embedding(i) for i in entities]
        self_vectors = entity_vectors[0].squeeze(1)  # (batch, dims)
        # (batch, -1, neigh, dims)
        neighbor_vectors = entity_vectors[1]  # (batch, neigh, dims)
        output, h0 = self.rnn(neighbor_vectors)
        # (batch, -1, dims)
        agg_vectors = output[:, -1, :]  # (batch, dims)
        agg_vectors = t.tanh(self.weight_agg(
            t.cat((self_vectors, agg_vectors), 1)))  # (batch, dims)
        agg_vectors = self.Dropout(agg_vectors)
        agg_vectors = F.normalize(agg_vectors)
        return agg_vectors

    # aggregate the item embeddings to obation user local preference
    def rs_attention(self, item_history_embedding, item_pre_embedding):
        # (batch, n_memories)
        item_pre_embedding = item_pre_embedding.unsqueeze(
            1).expand_as(item_history_embedding)
        logits = t.tanh(self.item_att(
            t.cat((item_history_embedding, item_pre_embedding), 2))).squeeze(2)
        attention = t.softmax(logits, 1)
        user_embedding = t.matmul(attention.unsqueeze(
            1), item_history_embedding).squeeze(1)  # (batch, dims)
        return user_embedding

    # user-specific GAT
    def SumAttention(self, self_vectors, neres, user_embeddings, neighbors, user_specific):
        # batch = len(self_vectors)
        user_embeddings = user_embeddings.unsqueeze(1)  # (batch, 1, dims)
        # (batch, -1, neighbor+1, dim)
        if user_specific:
            user_embeddings = user_embeddings.expand_as(
                neighbors)  # (batch, neighbor, dims)
            self_vectors1 = self_vectors.unsqueeze(1).expand_as(neighbors)
            # (batch, neighbor, 2dims)
            cat_vectors = t.cat((self_vectors1, neres), 2)
            trans_vectors = t.tanh(self.graph_att(cat_vectors))
            logits = t.sum(user_embeddings * trans_vectors,
                           2)  # (batch, neighbor)
            attention = t.softmax(logits, 1)  # (batch, neighbor)
            new_vector = t.matmul(attention.unsqueeze(
                1), neighbors).squeeze(1)  # (batch, dim)
        else:
            self_vectors1 = self_vectors.unsqueeze(1).expand_as(neighbors)
            cat_vectors = t.cat((self_vectors1, neres), 2)
            trans_vectors = t.tanh(self.graph_att(cat_vectors))
            logits = self.weight_nouser(trans_vectors).squeeze(-1)
            attention = t.softmax(logits, 1)
            new_vector = t.matmul(attention.unsqueeze(
                1), neighbors).squeeze(1)  # (batch, dim)
        new_vector = t.tanh(self.weight_agg(
            t.cat((self_vectors, new_vector), 1)))  # (batch, dims)
        return new_vector

Training

In [ ]:
def train(args, data_info, show_loss, show_topk):
    print('starting train...')
    train_data = data_info[0]
    eval_data = data_info[1]
    test_data = data_info[2]
    n_entity = data_info[3]
    n_relation = data_info[4]
    n_users = data_info[5]
    n_items = data_info[6]
    adj_enre = [data_info[8], data_info[9], data_info[11]]
    kg = data_info[7]
    user_history = data_info[10]
    weights_save_path = '../../result_w/' + args.dataset + '/cgat/'

    ensureDir(weights_save_path)

    model = CGAT(args, n_entity, n_relation, n_users, n_items)
    optimizer = optim.Adam(model.parameters(), lr=args.lr_rs,
                           weight_decay=args.l2_weight_rs)
    user_list, train_record, test_record, item_set, k_list = topk_settings(
        show_topk, train_data, test_data, n_items)
    stopping_step = 0
    should_stop = False
    cur_best_pre_0 = 0
    if args.use_cuda:
        model.cuda()
    for epoch in range(args.n_epochs):
        n = 0
        model.train()
        np.random.shuffle(train_data)
        np.random.shuffle(kg)
        t1 = time.time()
        loss_r = t.zeros(1)
        for user_indices, entity_lc, relation_lc, entity_gb, kg_indice in minibatch_rs(args, train_data, user_history, kg, adj_enre, True):
            user_indices = t.LongTensor(user_indices).cuda()
            entity_lc_cuda = []
            relation_lc_cuda = []
            entity_gb_cuda = []
            kg_cuda = []
            for i in range(len(entity_lc)):
                entity_lc_cuda.append([t.LongTensor(entity).cuda()
                                       for entity in entity_lc[i]])
                relation_lc_cuda.append(t.LongTensor(relation_lc[i]).cuda())
                entity_gb_cuda.append([t.LongTensor(entity).cuda()
                                       for entity in entity_gb[i]])
            for i in range(len(kg_indice)):
                kg_cuda.append(t.LongTensor(kg_indice[i]).cuda())

            score1, score2, all_loss = model(
                user_indices, entity_lc_cuda, relation_lc_cuda, entity_gb_cuda, kg_cuda)
            optimizer.zero_grad()
            all_loss.backward()
            optimizer.step()
            loss_r += (all_loss.cpu())
            n += 1
        t2 = time.time()
        if show_loss:
            print('time: {:.4f}'.format(t2 - t1))
            print('ave_loss: %.4f' % (loss_r.data.cpu().numpy() / n))
        # result used to tune hyperparameters
        # eval_auc = evaluation(
        #     model, eval_data, user_history, kg, adj_enre, args)
        # test_auc = evaluation(
        #     model, test_data, user_history, kg, adj_enre, args)
        # print('epoch %d    eval auc: %.4f  test auc: %.4f'
        #       % (epoch + 1, eval_auc, test_auc))

        # Top evaluation
        if show_topk and (epoch + 1) % 5 == 0:
            precision, recall, ndcg, hit, result, ave_time = topk_eval(
                args, model, user_list, user_history, train_record, test_record, item_set, k_list, adj_enre)
            print('precision: ', end='')
            for i in precision:
                print('%.4f\t' % i, end='')
            print()
            print('recall: ', end='')
            for i in recall:
                print('%.4f\t' % i, end='')
            print()
            print('ndcg: ', end='')
            for i in ndcg:
                print('%.4f\t' % i, end='')
            print()
            print('hit: ', end='')
            for i in hit:
                print('%.4f\t' % i, end='')
            print('\n')
            # print('averaged time of predicting for each user:', ave_time)
            cur_best_pre_0, stopping_step, should_stop = early_stopping(recall[1], cur_best_pre_0,
                                                                        stopping_step, expected_order='acc', flag_step=2)
            if should_stop is True:
                break
            # *********************************************************
            # save the user & item embeddings for pretraining.
            # if recall[3] == cur_best_pre_0:
                # t.save(model.state_dict(), weights_save_path + 'weights.pth')
                # pickle.dump(result, open(weights_save_path + 'results', 'wb'))
                # print('save the weights in path: ', weights_save_path)


In [ ]:
def ensureDir(dir_path):
    d = os.path.dirname(dir_path)
    if not os.path.exists(d):
        os.makedirs(d)

In [ ]:
def minibatch_rs(args, data, user_history, kg, adj_enre, train=True):
    adj_enlc, adj_relc, adj_engb = adj_enre[0], adj_enre[1], adj_enre[2]
    data_size = len(data)
    l1 = int(data_size / args.batch_size) + 1
    for i in range(l1):
        start = args.batch_size * i
        end = min(args.batch_size * (i + 1), data_size)
        user_indices = data[start:end, 0]  # (batch,)
        pos_item_indices = data[start:end, 1]  # (batch,)
        neg_item_indices = data[start:end, 2]

        user_his_batch = np.array(
            [user_history[user] for user in user_indices])  # (batch, n_memory)
        entity_his_lc, relation_his_lc = get_neighbors(
            args, np.reshape(user_his_batch, -1), adj_enlc, adj_relc)
        entity_his_gb = get_neighbors(
            args, np.reshape(user_his_batch, -1), adj_engb, None)

        # (batch, n_memory, 1), (batch, n_memory, n_neigh) the neighbors of user historical items
        entity_his_lc = [np.reshape(
            entity, [len(user_indices), args.n_memory, -1]) for entity in entity_his_lc]
        relation_his_lc = np.reshape(
            relation_his_lc, [len(user_indices), args.n_memory, args.n_neighbor])
        entity_his_gb = [np.reshape(
            entity, [len(user_indices), args.n_memory, -1]) for entity in entity_his_gb]

        # [(batch, 1),(batch, n_neigh)]; (batch, n_neigh)  the neighbor of candidate items
        pos_entity_lc, pos_relation_lc = get_neighbors(
            args, pos_item_indices, adj_enlc, adj_relc)
        neg_entity_lc, neg_relation_lc = get_neighbors(
            args, neg_item_indices, adj_enlc, adj_relc)
        pos_entity_gb = get_neighbors(
            args, pos_item_indices, adj_engb, None)
        neg_entity_gb = get_neighbors(
            args, neg_item_indices, adj_engb, None)

        entity_lc = (entity_his_lc, pos_entity_lc, neg_entity_lc)
        relation_lc = (relation_his_lc, pos_relation_lc, neg_relation_lc)
        entity_gb = (entity_his_gb, pos_entity_gb, neg_entity_gb)
        if train:
            if end > len(kg):
                start = len(kg) - args.batch_size
                end = len(kg)
            kg_de = kg[start:end]
            head_indices = kg_de[:, 0]
            relation_indices = kg_de[:, 1]
            tail_indices = kg_de[:, 2]
            tail_indices_ne = kg_de[:, 3]
        else:
            head_indices, relation_indices = None, None
            tail_indices, tail_indices_ne = None, None
        kg_indice = (head_indices, relation_indices,
                     tail_indices, tail_indices_ne)
        yield user_indices, entity_lc, relation_lc, entity_gb, kg_indice


In [ ]:
def evaluation(model, data, user_history, kg, adj_enre, args):
    model.eval()
    auc_list = []
    for user_indices, entity_lc, relation_lc, entity_gb, kg_indice in minibatch_rs(args, data, user_history, kg, adj_enre, False):
        user_indices = t.LongTensor(user_indices).cuda()
        entity_lc_cuda = []
        relation_lc_cuda = []
        entity_gb_cuda = []
        # kg_cuda = []
        for i in range(len(entity_lc)):
            entity_lc_cuda.append([t.LongTensor(entity).cuda()
                                   for entity in entity_lc[i]])
            relation_lc_cuda.append(t.LongTensor(relation_lc[i]).cuda())
            entity_gb_cuda.append([t.LongTensor(entity).cuda()
                                   for entity in entity_gb[i]])

        score1, score2, all_loss = model(
            user_indices, entity_lc_cuda, relation_lc_cuda, entity_gb_cuda, kg_indice)
        score1 = score1.data.cpu().numpy()
        score2 = score2.data.cpu().numpy()
        scores = np.concatenate((score1, score2))
        label1 = np.ones(len(score1))
        label2 = np.zeros(len(score2))
        labels = np.concatenate((label1, label2))
        auc = roc_auc_score(y_true=labels, y_score=scores)
        auc_list.append(auc)
    model.train()
    return float(np.mean(auc_list))

In [ ]:

def get_user_record(data):
    user_history_dict = dict()
    for interaction in data:
        user = interaction[0]
        item = interaction[1]
        if user not in user_history_dict:
            user_history_dict[user] = set()
        user_history_dict[user].add(item)
    return user_history_dict


In [ ]:
def topk_settings(show_topk, train_data, test_data, n_item):
    if show_topk:
        # user_num = 100
        k_list = [10, 20, 50]
        train_record = get_user_record(train_data)
        test_record = get_user_record(test_data)
        user_list = list(set(train_record.keys()) & set(test_record.keys()))
        # if len(user_list) > user_num:
        #     user_list = np.random.choice(
        #         user_list, size=user_num, replace=False)
        item_set = set(list(range(n_item)))
        return user_list, train_record, test_record, item_set, k_list
    else:
        return [None] * 5


In [ ]:
def topk_eval(args, model, user_list, user_history, train_record, test_record, item_set, k_list, adj_enre):
    precision_list = {k: [] for k in k_list}
    recall_list = {k: [] for k in k_list}
    ndcg_list = {k: [] for k in k_list}
    hit_list = {k: [] for k in k_list}
    result = dict()
    adj_enlc, adj_relc, adj_engb = adj_enre[0], adj_enre[1], adj_enre[2]
    all_time = 0
    for user in user_list:
        # time0 = time.time()
        test_item_list = list(item_set - set(train_record[user]))
        item_score_map = dict()
        start = 0
        while start < len(test_item_list):
            end = min(start + args.batch_size, len(test_item_list))
            item_indices = test_item_list[start:end]
            # user_indices = [user] * (end - start)
            user_indices = [user]
            user_his_batch = np.array(
                [user_history[user] for user in user_indices])  # (batch, n_memory)
            entity_his_lc, relation_his_lc = get_neighbors(
                args, np.reshape(user_his_batch, -1), adj_enlc, adj_relc)
            entity_his_gb = get_neighbors(
                args, np.reshape(user_his_batch, -1), adj_engb, None)

            # (batch, n_memory, 1), (batch, n_memory, n_neigh).
            entity_his_lc = [np.reshape(
                entity, [len(user_indices), args.n_memory, -1]) for entity in entity_his_lc]
            relation_his_lc = np.reshape(
                relation_his_lc, [len(user_indices), args.n_memory, args.n_neighbor])
            entity_his_gb = [np.reshape(
                entity, [len(user_indices), args.n_memory, -1]) for entity in entity_his_gb]
            # (batch, 1),(batch, n_neigh)
            pos_entity_lc, pos_relation_lc = get_neighbors(
                args, item_indices, adj_enlc, adj_relc)
            pos_entity_gb = get_neighbors(
                args, item_indices, adj_engb, None)

            entity_his_lc = [t.LongTensor(
                i).cuda() for i in entity_his_lc]
            pos_entity_lc = [t.LongTensor(i).cuda() for i in pos_entity_lc]
            relation_his_lc = t.LongTensor(relation_his_lc).cuda()
            pos_relation_lc = t.LongTensor(pos_relation_lc).cuda()
            user_indices = t.LongTensor(user_indices).cuda()
            entity_his_gb = [t.LongTensor(
                i).cuda() for i in entity_his_gb]
            pos_entity_gb = [t.LongTensor(i).cuda() for i in pos_entity_gb]

            neg_entity_lc = None
            neg_relation_lc = None
            neg_entity_gb = None, None
            head_indices, relation_indices = None, None
            tail_indices, tail_indices_ne = None, None
            entity_lc_cuda = [entity_his_lc, pos_entity_lc, neg_entity_lc]
            relation_lc_cuda = [relation_his_lc,
                                pos_relation_lc, neg_relation_lc]
            entity_gb_cuda = [entity_his_gb, pos_entity_gb, neg_entity_gb]
            kg_cuda = [head_indices, relation_indices,
                       tail_indices, tail_indices_ne]
            time1 = time.time()
            score1, score2, all_loss = model(
                user_indices, entity_lc_cuda, relation_lc_cuda, entity_gb_cuda, kg_cuda)
            all_time += (time.time() - time1)
            for item, score in zip(item_indices, list(score1.data.cpu().numpy())):
                item_score_map[item] = score
            start += args.batch_size
        item_score_pair_sorted = sorted(
            item_score_map.items(), key=lambda x: x[1], reverse=True)
        item_sorted = [i[0] for i in item_score_pair_sorted]
        label = np.zeros(len(item_sorted))
        for i, item in enumerate(item_sorted):
            if item in test_record[user]:
                label[i] = 1
        for k in k_list:
            hit_num = np.sum(label[:k])
            precision_list[k].append(hit_num / k)
            recall_list[k].append(hit_num / len(test_record[user]))
            ndcg_list[k].append(comp_ndcg(label, k))
            if np.sum(label[:k]) > 0:
                hit_list[k].append(1)
            else:
                hit_list[k].append(0)
    result['precision'] = np.array([precision_list[k] for k in k_list])
    result['recall'] = np.array([recall_list[k] for k in k_list])
    result['ndcg'] = np.array([ndcg_list[k] for k in k_list])
    result['hit'] = np.array([hit_list[k] for k in k_list])
    precision = [np.mean(precision_list[k]) for k in k_list]
    recall = [np.mean(recall_list[k]) for k in k_list]
    ndcg = [np.mean(ndcg_list[k]) for k in k_list]
    hit = [np.mean(hit_list[k]) for k in k_list]
    model.train()
    return precision, recall, ndcg, hit, result, all_time / len(user_list)

In [ ]:
def comp_ndcg(label, k):
    topk = label[:k]
    dcg = np.sum(topk / np.log2(np.arange(2, topk.size + 2)))
    dcg_max = np.sum(sorted(label, reverse=True)[
                     :k] / np.log2(np.arange(2, topk.size + 2)))
    if dcg_max == 0:
        return 0
    else:
        return dcg / dcg_max


In [ ]:
def get_neighbors(args, seeds, adj_entity, adj_relation):
    seeds_size = len(seeds)  # (batch, 1)
    seeds = np.expand_dims(seeds, axis=1)
    entities = [seeds]
    if adj_relation is not None:
        neighbor_entities = np.reshape(
            adj_entity[np.reshape(entities[0], -1)], [seeds_size, -1])
        neighbor_relations = np.reshape(
            adj_relation[np.reshape(entities[0], -1)], [seeds_size, -1])  # (batch, neighbor)
        entities.append(neighbor_entities)
        # relations.append(neighbor_relations)
        return entities, neighbor_relations
    else:
        neighbor_entities = np.reshape(
            adj_entity[np.reshape(entities[0], -1)], [seeds_size, -1])
        entities.append(neighbor_entities)
        return entities

In [ ]:
def early_stopping(log_value, best_value, stopping_step, expected_order='acc', flag_step=2):
    # early stopping strategy:
    assert expected_order in ['acc', 'dec']

    if (expected_order == 'acc' and log_value >= best_value) or (expected_order == 'dec' and log_value <= best_value):
        stopping_step = 0
        best_value = log_value
    else:
        stopping_step += 1

    if stopping_step >= flag_step:
        print("Early stopping is trigger at step: {} log:{}".format(
            flag_step, log_value))
        should_stop = True
    else:
        should_stop = False
    return best_value, stopping_step, should_stop

In [ ]:
model=train(args, data_info, show_loss, show_topk)

In [ ]:
def predict(model, user_id, item_ids):
    user_ids = [user_id] * len(item_ids)
    user_tensor = t.LongTensor(user_ids).cuda()
    item_lc_tensor = t.LongTensor(item_ids).cuda()
    item_gb_tensor = t.LongTensor(item_ids).cuda()

    with t.no_grad():
        model.eval()
        scores = model(user_tensor, ([], [item_lc_tensor], []), ([], [item_gb_tensor], []), None)

    return scores.cpu().numpy()


In [ ]:
predict(model, 1, 3);

In [ ]:
import torch as t


# Define the user ID and item IDs for which you want to make recommendations
user_id = 1
item_ids = [10, 20, 30, 40, 50]

# Call the predict function to get the predicted scores
scores = predict(model, user_id, item_ids)

# Print the predicted scores
print(scores)